## Load Dataset

In [20]:
import pandas as pd

df = pd.read_csv("logs.csv")

df.head()

,Unnamed: 0,timestamp,source,log_message,target_label
0,0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status
1,1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error
2,2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert
3,3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status
4,4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    2410 non-null   int64 
 1   timestamp     2410 non-null   object
 2   source        2410 non-null   object
 3   log_message   2410 non-null   object
 4   target_label  2410 non-null   object
dtypes: int64(1), object(4)
memory usage: 94.3+ KB


In [22]:
df.shape

(2410, 5)

In [13]:
# Unique values in "source" column

df["source"].unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [15]:
# Unique values in "target_label" column

df["target_label"].unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

## Clustering

In [16]:
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer

d:\01_CS\AI ML\Projects\Log Classification System\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# higher embedding quality, larger vector size
# great for classification and clustering

model_name = "sentence-transformers/all-mpnet-base-v2"

model = SentenceTransformer(model_name)

In [18]:
embeddings = model.encode(df['log_message'].tolist())

embeddings.shape

(2410, 768)

In [19]:
embeddings[:5]

array([[ 0.02419545,  0.02116937,  0.01027383, ...,  0.01493055,
        -0.03846572, -0.02089545],
       [ 0.04208487, -0.05253804,  0.00988673, ..., -0.004339  ,
         0.00253948,  0.03423762],
       [ 0.01313045,  0.01288181,  0.03380114, ...,  0.0383439 ,
        -0.00473739,  0.01573821],
       [ 0.02107986,  0.01952131,  0.01522476, ...,  0.0171412 ,
        -0.03507104, -0.02008633],
       [ 0.02350755,  0.01676207,  0.00847807, ...,  0.01490392,
        -0.04243018, -0.02468613]], shape=(5, 768), dtype=float32)

In [63]:
# eps : The maximum distance between two samples for one to be considered as in the neighborhood of the other
# min_sampples : The number of samples in a neighborhood for a point to be considered as a core point. This includes the point itself.

clustering = DBSCAN(eps=0.20, min_samples=3, metric="cosine").fit(embeddings)

df["cluster"] = clustering.labels_

In [64]:
df.head()

,Unnamed: 0,timestamp,source,log_message,target_label,cluster
0,0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,0
1,1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,1
2,2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,2
3,3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,0
4,4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,0


In [65]:
# clusters

df["cluster"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 31, 18, 19, 20, 21, -1, 22, 23, 24, 25, 26, 27, 28, 41, 29, 30,
       54, 32, 33, 34, 35, 36, 37, 38, 39, 52, 40, 42, 43, 46, 44, 45, 47,
       48, 49, 50, 51, 53])

In [66]:
df[df["cluster"] == 1]

,Unnamed: 0,timestamp,source,log_message,target_label,cluster
1,1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,1
10,10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,1
217,217,1/22/2025 5:45,BillingSystem,Mail service encountered a delivery glitch,Error,1
248,248,5/2/2025 23:04,ModernHR,Service disruption caused by email sending error,Critical Error,1
265,265,3/30/2025 23:53,ModernCRM,Email system had a problem sending emails,Error,1
361,361,11/19/2025 23:06,BillingSystem,Email service experienced a sending issue,Error,1
450,450,10/27/2025 5:59,ThirdPartyAPI,Email delivery system encountered an error,Error,1
477,477,12/2/2025 10:30,AnalyticsEngine,Email transmission error caused service impact,Critical Error,1
570,570,11/7/2025 18:08,ThirdPartyAPI,Email service impacted by sending failure,Critical Error,1
678,678,4/28/2025 15:13,AnalyticsEngine,Email delivery problem affected system,Critical Error,1


In [74]:
cluster_counts = df["cluster"].value_counts()
larger_clusters = cluster_counts[cluster_counts > 10].index

for cluster in larger_clusters:
    print(f"Cluster : {cluster}")
    print(df[df["cluster"] == cluster]["log_message"].to_string())
    print()

Cluster : 0
0       nova.osapi_compute.wsgi.server [req-b9718cd8-f...
3       nova.osapi_compute.wsgi.server [req-4895c258-b...
4       nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...
5       nova.osapi_compute.wsgi.server [req-f0bffbc3-5...
9       nova.osapi_compute.wsgi.server [req-2bf7cfee-a...
12      nova.osapi_compute.wsgi.server [req-d4f8d0c2-4...
16      nova.osapi_compute.wsgi.server [req-6fe0e366-f...
17      nova.osapi_compute.wsgi.server [req-5f1c2027-e...
19      nova.osapi_compute.wsgi.server [req-945d1f31-a...
20      nova.osapi_compute.wsgi.server [req-033d97b9-6...
21      nova.osapi_compute.wsgi.server [req-75bc6269-8...
23      nova.osapi_compute.wsgi.server [req-077c3c87-b...
24      nova.osapi_compute.wsgi.server [req-4e83daf7-a...
25      nova.osapi_compute.wsgi.server [req-bfce366e-9...
28      nova.osapi_compute.wsgi.server [req-5e6e042b-f...
31      nova.metadata.wsgi.server [-] 10.11.21.138,10....
33      nova.osapi_compute.wsgi.server [req-fe9ef402-d...
37

Apply regex on 11, 13, 8, 19, 4, 27, 15, 9 clusters

In [97]:
df[df["cluster"] == 13]["log_message"].head(20)

30     Backup started at 2025-05-14 07:06:55.
44     Backup started at 2025-02-15 20:00:19.
108      Backup ended at 2025-08-08 13:06:23.
128    Backup started at 2025-11-14 08:27:43.
191    Backup started at 2025-12-09 10:19:11.
209    Backup started at 2025-01-15 04:36:39.
236    Backup started at 2025-08-29 15:17:18.
237      Backup ended at 2025-06-01 06:27:38.
264      Backup ended at 2025-11-16 08:17:13.
325      Backup ended at 2025-07-18 17:06:54.
367      Backup ended at 2025-08-18 14:29:26.
382      Backup ended at 2025-02-21 07:53:56.
407      Backup ended at 2025-08-22 14:11:44.
429    Backup started at 2025-10-12 10:35:15.
458      Backup ended at 2025-08-08 14:14:05.
483    Backup started at 2025-04-19 23:43:04.
518      Backup ended at 2025-04-16 05:59:10.
535      Backup ended at 2025-07-01 14:07:11.
618      Backup ended at 2025-07-31 13:19:58.
634    Backup started at 2025-10-31 23:48:54.
Name: log_message, dtype: object

**Patterns**

- User Action
    - User User900 logged in
    - User User272 logged out
    - Account with ID 5351 created by User634

- System Notification
    - Backup started at 2025-08-29 15:17:18
    - Backup ended at 2025-06-01 06:27:38
    - Backup completed successfully
    - System updated to version 3.9.1
    - File data_6169.csv uploaded successfully by user User953
    - Disk cleanup completed successfully
    - System reboot initiated by user User243


## Classification : Regex

In [98]:
import re

def classify_with_regex(log_message):
    '''
        input : Log message
        output : Error type (target_label)
    '''
    regex_patterns = {
        r"User User\d+ logged (in|out)." : "User Action",
        r"Backup (started|ended) at .*" : "System Notification",
        r"Backup completed successfully." : "System Notification",
        r"System updated to version .*" : "System Notification",
        r"File .* uploaded successfully by user .*" : "System Notification",
        r"Disk cleanup completed successfully." : "System Notification",
        r"System reboot initiated by user .*" : "System Notification",
        r"Account with ID .* created by .*" : "User Action"
    }

    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message):
            return label
        
    return None

In [101]:
classify_with_regex("User User23 logged in.")

'User Action'

In [102]:
classify_with_regex("System reboot initiated by user User23.")

'System Notification'

In [103]:
classify_with_regex("Hey you, come here.")

In [107]:
# Apply regex classification on our training samples

df["regex_label"] = df["log_message"].apply(classify_with_regex)

In [108]:
# "regex_label" != None

df[df["regex_label"].notnull()].head(10)

,Unnamed: 0,timestamp,source,log_message,target_label,cluster,regex_label
7,7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,4,System Notification
14,14,1/4/2025 1:43,ThirdPartyAPI,File data_3847.csv uploaded successfully by us...,System Notification,4,System Notification
15,15,5/1/2025 9:41,ModernCRM,Backup completed successfully.,System Notification,8,System Notification
18,18,2/22/2025 17:49,ModernCRM,Account with ID 5351 created by User634.,User Action,9,User Action
27,27,9/24/2025 19:57,ThirdPartyAPI,User User685 logged out.,User Action,11,User Action
30,30,4/26/2025 7:54,AnalyticsEngine,Backup started at 2025-05-14 07:06:55.,System Notification,13,System Notification
36,36,11/19/2025 13:14,BillingSystem,System reboot initiated by user User243.,System Notification,15,System Notification
44,44,6/20/2025 0:53,BillingSystem,Backup started at 2025-02-15 20:00:19.,System Notification,13,System Notification
48,48,7/1/2025 9:43,ModernHR,File data_7366.csv uploaded successfully by us...,System Notification,4,System Notification
50,50,1/16/2025 21:45,ThirdPartyAPI,System updated to version 3.9.1.,System Notification,19,System Notification


In [109]:
# "regex_label" == None

df[df["regex_label"].isnull()].head(10)

,Unnamed: 0,timestamp,source,log_message,target_label,cluster,regex_label
0,0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,0,None
1,1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,1,None
2,2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,2,None
3,3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,0,None
4,4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,0,None
5,5,2025-10-09 10:30:31,ModernHR,nova.osapi_compute.wsgi.server [req-f0bffbc3-5...,HTTP Status,0,None
6,6,3/1/2025 19:14,ModernHR,Shard 6 replication task ended in failure,Error,3,None
8,8,2025-02-12 10:42:29,ThirdPartyAPI,nova.compute.claims [req-a07ac654-8e81-416d-bf...,Resource Usage,5,None
9,9,2025-03-30 04:01:45,ModernHR,nova.osapi_compute.wsgi.server [req-2bf7cfee-a...,HTTP Status,0,None
10,10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,1,None


## Classification : Embeddings

In [110]:
# extract training samples for embedding approach

df_non_regex = df[df['regex_label'].isnull()].copy()

df_non_regex.shape

(1910, 7)

In [114]:
df_non_regex["target_label"].value_counts()

target_label
HTTP Status            1017
Security Alert          371
Error                   177
Resource Usage          177
Critical Error          161
Workflow Error            4
Deprecation Warning       3
Name: count, dtype: int64

- **Workflow Error** and **Deprecation Warning** do not have enough training samples
- we will separate them for LLM call

In [116]:
# Deprecation Warning 

df[df["target_label"] == "Deprecation Warning"]

,Unnamed: 0,timestamp,source,log_message,target_label,cluster,regex_label
255,255,2025-05-03 16:55:35,LegacyCRM,API endpoint 'getCustomerDetails' is deprecate...,Deprecation Warning,-1,None
1734,1734,2025-04-30 07:47:30,LegacyCRM,The 'ExportToCSV' feature is outdated. Please ...,Deprecation Warning,-1,None
1826,1826,2025-01-23 10:33:36,LegacyCRM,Support for legacy authentication methods will...,Deprecation Warning,-1,None


In [117]:
# Workflow Error

df[df["target_label"] == "Workflow Error"]

,Unnamed: 0,timestamp,source,log_message,target_label,cluster,regex_label
60,60,2025-10-06 16:55:23,LegacyCRM,Lead conversion failed for prospect ID 7842 du...,Workflow Error,-1,None
377,377,2025-06-24 12:16:29,LegacyCRM,Customer follow-up process for lead ID 5621 fa...,Workflow Error,-1,None
1325,1325,2025-04-17 07:33:44,LegacyCRM,Escalation rule execution failed for ticket ID...,Workflow Error,-1,None
2217,2217,2025-05-12 09:46:54,LegacyCRM,Task assignment for TeamID 3425 could not comp...,Workflow Error,-1,None


**Observation** : Both belongs to source = "LegacyCRM"

In [118]:
df_non_legacy = df_non_regex[df_non_regex["source"] != "LegacyCRM"]

df_legacy = df_non_regex[df_non_regex["source"] == "LegacyCRM"]

In [119]:
df_non_legacy.shape

(1903, 7)

In [120]:
embeddings_non_legacy = model.encode(df_non_legacy["log_message"].tolist())

embeddings_non_legacy.shape

(1903, 768)

In [121]:
X = embeddings_non_legacy

y = df_non_legacy['target_label'].values

In [122]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [123]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

clf = LogisticRegression(max_iter=1000)

clf.fit(X_train, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [126]:
y_pred = clf.predict(X_test)

report = classification_report(y_test, y_pred)

print(report)

                precision    recall  f1-score   support

Critical Error       0.92      1.00      0.96        48
         Error       1.00      0.91      0.96        47
   HTTP Status       1.00      1.00      1.00       304
Resource Usage       1.00      1.00      1.00        49
Security Alert       1.00      1.00      1.00       123

      accuracy                           0.99       571
     macro avg       0.98      0.98      0.98       571
  weighted avg       0.99      0.99      0.99       571



In [127]:
# Save the model using joblib

import joblib

joblib.dump(clf, '../models/log_classifier.joblib')

['../models/log_classifier.joblib']